In [19]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Dropdown, fixed
from IPython.display import display
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

In [2]:
# Path to go up one level from the current folder to the parent folder
parent_path = os.path.join(os.getcwd(), '..')

# Path to the target data folder
data_folder_path = os.path.join(parent_path, 'Data')

In [3]:
menu_df = pd.read_csv(os.path.join(data_folder_path, 'menu_analysis.csv'))
customers_df = pd.read_csv(os.path.join(data_folder_path, 'customers.csv'))
stores_df = pd.read_csv(os.path.join(data_folder_path, 'stores.csv'))
structure_df = pd.read_csv(os.path.join(data_folder_path, 'structure.csv'))
size_df = pd.read_csv(os.path.join(data_folder_path, 'order_size.csv'))
transactions_df = pd.read_csv(
        os.path.join(data_folder_path, "transactions.csv"),
        parse_dates=['Transaction_Date']  # Modify as needed if format issues arise
    )

In [4]:
df_list = [menu_df, customers_df, stores_df, structure_df, size_df, transactions_df]
for dataframe in df_list:
    print(dataframe.head())

     Group Item_Number Ingredient  Price  Cost  Points Earn  Points Redeem
0  Primary      Item_1    Chicken    6.5   2.0         65.0          400.0
1  Primary      Item_2       Beef    8.0   2.5         80.0          500.0
2  Primary      Item_3     Shrimp    7.5   3.0         75.0          600.0
3  Primary      Item_4       Tofu    5.5   1.0         55.0          200.0
4  Primary      Item_5  Chickpeas    4.5   0.5         45.0          100.0
  Customer_ID First_Name Last_Name                   Email  \
0    CUST0000     Sheila    Garcia    perryamy@example.org   
1    CUST0001    Matthew   Simpson      uwalls@example.net   
2    CUST0002    Michael     Curry    mcoleman@example.com   
3    CUST0003       Tara    Forbes  westdaniel@example.net   
4    CUST0004     Leslie       Lee     amber03@example.net   

                   Phone         DOB  Gender                        Address  \
0      691-567-7618x9089  1994-09-15   Other          10471 Vasquez Squares   
1   001-857-509-518

In [7]:
transactions_df.columns.tolist()

['Order_Number',
 'Transaction_Date',
 'Transaction_Time',
 'Store_Number',
 'Customer_ID',
 'Item_Number']

In [12]:
menu_df.columns.to_list()

['Group',
 'Item_Number',
 'Ingredient',
 'Price',
 'Cost',
 'Points Earn',
 'Points Redeem']

### Modification 1

In [10]:
transactions_df['Transaction_Date'] = pd.to_datetime(transactions_df['Transaction_Date'])

### Modification 2

In [13]:
working_df = pd.merge(transactions_df, menu_df[['Item_Number', 'Price']])

### Modification 3

In [14]:
monthly_revenue = working_df.groupby([working_df['Transaction_Date'].dt.to_period('M'), 'Store_Number'])['Price'].sum().reset_index()

### Modification 4

In [16]:
# Assuming transactions_df has columns: 'Date', 'Store_ID', 'Total_Amount'
monthly_revenue['Transaction_Date'] = monthly_revenue['Transaction_Date'].dt.to_timestamp()

In [17]:
monthly_revenue.head()

,Transaction_Date,Store_Number,Price
0,2023-06-01,STR_1,4147.0
1,2023-06-01,STR_10,3991.5
2,2023-06-01,STR_2,4806.0
3,2023-06-01,STR_3,3738.5
4,2023-06-01,STR_4,4309.0


# 1. Monthly Revenue Trend Chart

In [20]:
fig = go.Figure()

for store in monthly_revenue['Store_Number'].unique():
    store_data = monthly_revenue[monthly_revenue['Store_Number'] == store]
    fig.add_trace(go.Scatter(
        x=store_data['Transaction_Date'],
        y=store_data['Price'],
        mode='lines+markers',
        name=f'Store {store}'
    ))

fig.update_layout(
    title='Monthly Revenue Trend by Store',
    xaxis_title='Date',
    yaxis_title='Revenue ($)',
    legend_title='Store',
    hovermode='x unified'
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

fig.show()